In [2]:
import boto3

In [3]:
session = boto3.Session()
credentials = session.get_credentials()

# Credentials are refreshable, so accessing your access key / secret key
# separately can lead to a race condition. Use this to get an actual matched
# set.
credentials = credentials.get_frozen_credentials()
access_key = credentials.access_key
secret_key = credentials.secret_key
region = session.get_available_regions("ec2")
session.region_name
#session.get_available_services()

'us-east-2'

In [31]:
ec2_client = boto3.client('ec2')
ec2_res = boto3.resource('ec2')

In [32]:
def list_instances(ec2_res):
    for instance_1 in ec2_res.instances.all():
        print(
         "Id: {0}\nPlatform: {1}\nType: {2}\nPublic IPv4: {3}\nAMI: {4}\nState: {5}\n".format(
         instance_1.id, instance_1.platform, instance_1.instance_type, instance_1.public_ip_address, instance_1.image.id, instance_1.state
         )
     )

In [33]:
def instance_state_checks(ec2_res,state):
    # create filter for instances in running state
    filters = [
        {
            'Name': 'instance-state-name', 
            'Values': [state]
        }
    ]
    
    # filter the instances
    instances = ec2_res.instances.filter(Filters=filters)

    RunningInstances = []

    for instance in instances:
        # for each instance, append to array and print instance id
        RunningInstances.append(instance.id)
        print(instance.id)

In [34]:
instance_state_checks(ec2_res,"stopped")

i-0da4f7758b48ce959
i-00f25c1aafffbf5dc


In [35]:
instance_state_checks(ec2_res,"running")

In [36]:
instance_state_checks(ec2_res,"terminated")

i-030d02cf812d65158


In [37]:
for my_instance in ec2_res.instances.all():
     print(
         "Id: {0}\nPlatform: {1}\nType: {2}\nPublic IPv4: {3}\nAMI: {4}\nState: {5}\n".format(
         my_instance.id, my_instance.platform, my_instance.instance_type, 
             my_instance.public_ip_address, my_instance.image.id, my_instance.state
         )
     )

Id: i-0da4f7758b48ce959
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-00f25c1aafffbf5dc
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-030d02cf812d65158
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 48, 'Name': 'terminated'}



---

### Create and run new Instance

Using same Image ID for all Instances

In [40]:
new_instance = ec2_client.run_instances(ImageId=my_instance.image.id,
                                InstanceType='t2.micro',
                                MinCount=1,
                                MaxCount=1, 
                                InstanceInitiatedShutdownBehavior='terminate', 
                            )

In [41]:
new_instance['Instances'][0]

{'AmiLaunchIndex': 0,
 'ImageId': 'ami-0277b52859bac6f4b',
 'InstanceId': 'i-0275ca887da5eb3a8',
 'InstanceType': 't2.micro',
 'LaunchTime': datetime.datetime(2021, 7, 1, 15, 39, 23, tzinfo=tzutc()),
 'Monitoring': {'State': 'disabled'},
 'Placement': {'AvailabilityZone': 'us-east-2a',
  'GroupName': '',
  'Tenancy': 'default'},
 'PrivateDnsName': 'ip-172-31-13-1.us-east-2.compute.internal',
 'PrivateIpAddress': '172.31.13.1',
 'ProductCodes': [],
 'PublicDnsName': '',
 'State': {'Code': 0, 'Name': 'pending'},
 'StateTransitionReason': '',
 'SubnetId': 'subnet-0f4d9464',
 'VpcId': 'vpc-046cdd6f',
 'Architecture': 'x86_64',
 'BlockDeviceMappings': [],
 'ClientToken': '',
 'EbsOptimized': False,
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'NetworkInterfaces': [{'Attachment': {'AttachTime': datetime.datetime(2021, 7, 1, 15, 39, 23, tzinfo=tzutc()),
    'AttachmentId': 'eni-attach-07ed572006310521a',
    'DeleteOnTermination': True,
    'DeviceIndex': 0,
    'Status': 'attaching'},
   'Des

In [42]:
list(new_instance.keys())
instances = new_instance['Instances'][0]

AMI = new_instance['Instances'][0]["ImageId"]
print(f'{AMI} {instances["InstanceId"]}')

ami-0277b52859bac6f4b i-0275ca887da5eb3a8


---

### Terminate Instance(s)

In [27]:
ec2_client.terminate_instances(InstanceIds=[instances["InstanceId"]])

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-030d02cf812d65158',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '71e9c4ac-771f-46af-962e-eed5b2ffad94',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '71e9c4ac-771f-46af-962e-eed5b2ffad94',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'vary': 'accept-encoding',
   'date': 'Thu, 01 Jul 2021 15:31:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

---

### We can always envoke the ```list_instances``` function at any time to see the info on our instances

In [43]:
list_instances(ec2_res)

Id: i-0da4f7758b48ce959
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-00f25c1aafffbf5dc
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 80, 'Name': 'stopped'}

Id: i-030d02cf812d65158
Platform: None
Type: t2.micro
Public IPv4: None
AMI: ami-0277b52859bac6f4b
State: {'Code': 48, 'Name': 'terminated'}

Id: i-0275ca887da5eb3a8
Platform: None
Type: t2.micro
Public IPv4: 3.142.43.44
AMI: ami-0277b52859bac6f4b
State: {'Code': 16, 'Name': 'running'}

